<a href="https://colab.research.google.com/github/JoelFausto/PIBIC_Codes/blob/main/%5BPIBIC%5D%20Ataque_de_Infer%C3%AAncia_de_Atributo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importando bibliotecas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install plotly --upgrade

In [ ]:
!pip install dask

In [ ]:
import pandas as pd
import dask.dataframe as dd

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
from collections import Counter

### Dados

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# Carregando dataset do Drive (O Colab não suporta upload de datasets muito grandes, por isso deve vir direto do Drive)
microdados = pd.read_parquet('/content/drive/MyDrive/PIBIC/dataCenso2018_colunasEspecificas.parquet')

### Ataque de Inferência de Atributo

#### Código baseado nos conceitos matemáticos (TED)

In [ ]:
def suc_prior_deterministic(data, sensitive_attr):
    """
    Sucesso determinístico a priori do adversário em um ataque ACU.
    """
    sensitive_values = data[sensitive_attr].unique()
    return 1 if len(sensitive_values) == 1 else 0

def suc_post_deterministic(data, qid_attrs, sensitive_attr):
    """
    Sucesso determinístico a posteriori do adversário em um ataque ACU.
    """
    groups = data.groupby(qid_attrs)
    total_records = len(data)
    success_count = 0

    for _, group in groups:
        if len(group[sensitive_attr].unique()) == 1:
            success_count += len(group)

    return success_count / total_records

def suc_prior_probabilistic(data, sensitive_attr):
    """
    Sucesso probabilístico a priori do adversário em um ataque ACU.
    """
    value_counts = data[sensitive_attr].value_counts()
    total_records = len(data)
    return value_counts.max() / total_records if total_records > 0 else 0

def suc_post_probabilistic(data, qid_attrs, sensitive_attr):
    """
    Sucesso probabilístico a posteriori do adversário em um ataque ACU.
    """
    groups = data.groupby(qid_attrs, dropna=False)  # Inclui NaN como grupo distinto
    total_records = len(data)
    total_success = 0

    for _, group in groups:
        group_size = len(group)
        if group_size > 0:
            value_counts = group[sensitive_attr].value_counts(dropna=False)  # Inclui NaN ao contar
            total_success += value_counts.max()

    return total_success / total_records if total_records > 0 else 0

def degradation_privacy_deterministic(data, qid_attrs, sensitive_attr):
    """
    Degradação determinística de privacidade em um ataque ACU.
    """
    prior = suc_prior_deterministic(data, sensitive_attr)
    post = suc_post_deterministic(data, qid_attrs, sensitive_attr)
    return post - prior

def degradation_privacy_probabilistic(data, qid_attrs, sensitive_attr):
    """
    Degradação probabilística de privacidade em um ataque ACU.
    """
    prior = suc_prior_probabilistic(data, sensitive_attr)
    post = suc_post_probabilistic(data, qid_attrs, sensitive_attr)
    return post / prior if prior > 0 else 0

##### IN_FINANCIAMENTO_ESTUDANTIL

In [ ]:
qid_attrs = ['CO_CURSO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

print("Sucesso determinístico a priori:", suc_prior_deterministic(data, sensitive_attr))
print("Sucesso determinístico a posteriori:", suc_post_deterministic(data, qid_attrs, sensitive_attr))
print("Degradação determinística de privacidade:", degradation_privacy_deterministic(data, qid_attrs, sensitive_attr))

print("Sucesso probabilístico a priori:", suc_prior_probabilistic(data, sensitive_attr))
print("Sucesso probabilístico a posteriori:", suc_post_probabilistic(data, qid_attrs, sensitive_attr))
print("Degradação probabilística de privacidade:", degradation_privacy_probabilistic(data, qid_attrs, sensitive_attr))

Sucesso determinístico a priori: 0
Sucesso determinístico a posteriori: 0.25159520777727556
Degradação determinística de privacidade: 0.25159520777727556
Sucesso probabilístico a priori: 0.4737083804702005
Sucesso probabilístico a posteriori: 0.8288100901984822
Degradação probabilística de privacidade: 1.7496209152470759


In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

print("Sucesso determinístico a priori:", suc_prior_deterministic(data, sensitive_attr))
print("Sucesso determinístico a posteriori:", suc_post_deterministic(data, qid_attrs, sensitive_attr))
print("Degradação determinística de privacidade:", degradation_privacy_deterministic(data, qid_attrs, sensitive_attr))

print("Sucesso probabilístico a priori:", suc_prior_probabilistic(data, sensitive_attr))
print("Sucesso probabilístico a posteriori:", suc_post_probabilistic(data, qid_attrs, sensitive_attr))
print("Degradação probabilística de privacidade:", degradation_privacy_probabilistic(data, qid_attrs, sensitive_attr))

Sucesso determinístico a priori: 0
Sucesso determinístico a posteriori: 0.31525201494209787
Degradação determinística de privacidade: 0.31525201494209787
Sucesso probabilístico a priori: 0.4737083804702005
Sucesso probabilístico a posteriori: 0.8540614845109434
Degradação probabilística de privacidade: 1.80292669440048


In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

Sucesso determinístico a priori: 0
Sucesso determinístico a posteriori: 0.7227774128919482
Degradação determinística de privacidade: 0.7227774128919482
Sucesso probabilístico a priori: 0.4737083804702005
Sucesso probabilístico a posteriori: 0.9138462471931771
Degradação probabilística de privacidade: 1.9291325314660848


In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

Sucesso determinístico a priori: 0
Sucesso determinístico a posteriori: 0.9241651629578265
Degradação determinística de privacidade: 0.9241651629578265
Sucesso probabilístico a priori: 0.4737083804702005
Sucesso probabilístico a posteriori: 0.9710487836167835
Degradação probabilística de privacidade: 2.049887280129867


In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

##### IN_DEFICIENCIA

In [ ]:
qid_attrs = ['CO_CURSO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

print("Sucesso determinístico a priori:", suc_prior_deterministic(data, sensitive_attr))
print("Sucesso determinístico a posteriori:", suc_post_deterministic(data, qid_attrs, sensitive_attr))
print("Degradação determinística de privacidade:", degradation_privacy_deterministic(data, qid_attrs, sensitive_attr))

print("Sucesso probabilístico a priori:", suc_prior_probabilistic(data, sensitive_attr))
print("Sucesso probabilístico a posteriori:", suc_post_probabilistic(data, qid_attrs, sensitive_attr))
print("Degradação probabilística de privacidade:", degradation_privacy_probabilistic(data, qid_attrs, sensitive_attr))

Sucesso determinístico a priori: 0
Sucesso determinístico a posteriori: 0.15459301541002116
Degradação determinística de privacidade: 0.15459301541002116
Sucesso probabilístico a priori: 0.9403696085343882
Sucesso probabilístico a posteriori: 0.9732570597083633
Degradação probabilística de privacidade: 1.0349728988213813


In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

print("Sucesso determinístico a priori:", suc_prior_deterministic(data, sensitive_attr))
print("Sucesso determinístico a posteriori:", suc_post_deterministic(data, qid_attrs, sensitive_attr))
print("Degradação determinística de privacidade:", degradation_privacy_deterministic(data, qid_attrs, sensitive_attr))

print("Sucesso probabilístico a priori:", suc_prior_probabilistic(data, sensitive_attr))
print("Sucesso probabilístico a posteriori:", suc_post_probabilistic(data, qid_attrs, sensitive_attr))
print("Degradação probabilística de privacidade:", degradation_privacy_probabilistic(data, qid_attrs, sensitive_attr))

Sucesso determinístico a priori: 0
Sucesso determinístico a posteriori: 0.45221248915863616
Degradação determinística de privacidade: 0.45221248915863616
Sucesso probabilístico a priori: 0.9403696085343882
Sucesso probabilístico a posteriori: 0.9784216047188571
Degradação probabilística de privacidade: 1.0404649361688483


In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

Sucesso determinístico a priori: 0
Sucesso determinístico a posteriori: 0.6583656759068338
Degradação determinística de privacidade: 0.6583656759068338
Sucesso probabilístico a priori: 0.9403696085343882
Sucesso probabilístico a posteriori: 0.9749169433833158
Degradação probabilística de privacidade: 1.0367380384642282


In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

Sucesso determinístico a priori: 0
Sucesso determinístico a posteriori: 0.9454647836153036
Degradação determinística de privacidade: 0.9454647836153036
Sucesso probabilístico a priori: 0.9403696085343882
Sucesso probabilístico a posteriori: 0.9891900376271747
Degradação probabilística de privacidade: 1.0519162132099054


In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)